# Analysis of S&P 500 

Risk-free rate is assumed to be zero and the portfolios weights is therefor entirely in risky assets.

In [93]:
import yfinance as yf
import pandas as pd
import numpy as np
import sqlite3
from scipy.optimize import minimize

Import tickers

In [37]:
# # Load the current list of S&P 500 companies
# sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# sp500_table = pd.read_html(sp500_url, header=0)[0]

# # Extracting the ticker symbols 
# sp500_tickers = sp500_table["Symbol"].tolist()

# # Define the time period
# start_date = '2000-01-01'
# end_date = '2022-12-31'

# # Building a dataframe with stock prices for the companies
# df = yf.download(sp500_tickers,start=start_date,end=end_date)
# df = df['Adj Close']

[*********************100%%**********************]  503 of 503 completed


4 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 -> 2022-12-31)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Save data in sql database

In [42]:
# # Seting up database for later use 
# SNP500_data = sqlite3.connect(database="SNP500_data.sqlite")

# (df.to_sql(name="SNP500_prices", 
#           con=SNP500_data,  
#           if_exists="replace",
#           index = True)
# )

5787

Import data from sql database

In [43]:
SNP500_data = sqlite3.connect(database="SNP500_data.sqlite")

prices = (pd.read_sql_query(sql="SELECT * FROM SNP500_prices", con=SNP500_data, index_col="Date"))

In [86]:
tickers=["MMM","AOS","ABT","ADM","ADBE","ADP","AES","AFL","A","AKAM"]

prices = prices[tickers]
# Change the index column to be the date
prices.index = pd.to_datetime(prices.index)

# Extract the stock price at the end of each month from downloaded data
prices_monthly = prices.resample("1M").last()
prices_monthly.index = prices_monthly.index.date

In [87]:
# Drop stocks with missing values
prices_monthly = prices_monthly.dropna(how="all",axis=1)

In [88]:
# Computing log returns using np.log and shift, as well as removing the first row (since its NaN)
ret = np.log(prices_monthly / prices_monthly.shift(1))[1:]

### Performance of Mean-Variance Ptf vs. Global Minimum Ptf

In [89]:
# Optimal weights of MVP (tangency) portfolio without constraints:
def MVP_w(returns):
    ones = np.ones(len(returns.columns)) 
    mu = returns.mean()
    var_cov = returns.cov()
    numerator = np.linalg.inv(var_cov) @ mu
    denominator = ones.T @ np.linalg.inv(var_cov) @ mu
    optimal_weight = numerator/denominator
    return optimal_weight

# Optimal weights of GMV portfolio without constraints:
def GMV_w(returns):
    ones = np.ones(len(returns.columns)) 
    var_cov = returns.cov()
    numerator = np.linalg.inv(var_cov) @ ones
    denominator = ones.T @ np.linalg.inv(var_cov) @ ones
    optimal_weight = numerator/denominator
    return optimal_weight

In [90]:
MVP_weights = pd.DataFrame(index = ret.index, columns =ret.columns)

for i in ret.index[60:]: # require 5y of data 
    start_date = i - pd.DateOffset(months = 60)
    end_date = i-pd.DateOffset(months = 1)
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()
    df = ret.loc[start_date:end_date]
    MVP_weights.loc[i] = MVP_w(df) 

MVP_weights = MVP_weights.dropna()

In [91]:
# GMV portfolio:
GMV_weights = pd.DataFrame(index = ret.index, columns =ret.columns)

for i in ret.index[60:]: 
    start_date = i - pd.DateOffset(months = 60)
    end_date = i - pd.DateOffset(months = 1)
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()
    df = ret.loc[start_date:end_date]
    GMV_weights.loc[i] = GMV_w(df) 

GMV_weights = GMV_weights.dropna()

In [92]:
# Return on MVP and GMV portfolio without constraints:
MVP_return = (MVP_weights * ret.iloc[60:]).sum(axis = 1)
GMV_return = (GMV_weights * ret.iloc[60:]).sum(axis = 1)

In [82]:
def sharpe_ratio(return_):             
    Rf=0
    m_mean_return = return_.mean()     
    m_vol = return_.std()              
    y_mean_return = m_mean_return * 12  
    y_vol = m_vol * np.sqrt(12)         
    SR = (y_mean_return-Rf)/y_vol
    return SR

In [83]:
# Sharpe ratio for MVP portfolio:
print(f'Annualized Sharpe ratio of MVP portfolio is {sharpe_ratio(MVP_return)}.')
# Sharpe ratio for GMV portfolio:
print(f'Annualized Sharpe Ratio of GMV portfolio is {sharpe_ratio(GMV_return)}.')

Annualized Sharpe ratio of MVP portfolio is -0.14067935252818148.
Annualized Sharpe Ratio of GMV portfolio is 0.6918133195127342.


### Constrained Mean-Variance Ptf

In [98]:
# Define objective to be minimized (portfolio variance)
def objective_MVP(weights, mu, V):
    portfolio_return = weights.T @ mu
    portfolio_variance = weights.T @ V @ weights
    sharpe = portfolio_return/np.sqrt(portfolio_variance)
    return -sharpe

In [99]:
# Calculate the optimal weight of MVP-C portfolio:
def MVP_C_w(returns):
    mu = returns.mean()
    V = returns.cov()

    # Define the constraint that the weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)- 1})

    # Define the bounds for each weight (0 <= weight <= 999)
    bounds = tuple((0, 999) for _ in range(len(V)))

    # Define the initial guess for weights
    initial_weights = np.ones(len(V)) / len(V)

    # Use scipy.optimize.minimize to find the optimal weights
    result = minimize(objective_MVP, initial_weights, args = (mu,V,), method = 'SLSQP',bounds = bounds, constraints = constraints)

    # Extract the optimal weights
    optimal_weights = result.x
    return optimal_weights

In [100]:
MVP_C_weights = pd.DataFrame(index=ret.index, columns=ret.columns)
for i in ret.index[60:]:    
    start_date = i - pd.DateOffset(months=60)
    end_date = i-pd.DateOffset(months=1)
    start_date = pd.to_datetime(start_date).date()
    end_date=pd.to_datetime(end_date).date()
    df = ret.loc[start_date:end_date]

    # calculate MVP-C portfolio for each rolling window
    MVP_C_weights.loc[i] = MVP_C_w(df)
    
MVP_C_weights = MVP_C_weights.dropna()

### Constrained Global Minimum Ptf

In [101]:
# Define the objective to be minimized (portfolio variance)
def objective_GMV(weights, V):
    portfolio_variance = weights.T @ V @ weights
    return portfolio_variance

In [102]:
# Calculate the optimal weight of GMV-C portfolio:
def GMV_C_w(returns):
    V = returns.cov()
    # Define the constraint that the weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # Define the bounds for each weight (0 <= weight <= 999)
    bounds = tuple((0, 999) for _ in range(len(V)))

    # Define the initial guess for the weights
    initial_weights = np.ones(len(V)) / len(V)

    # Use scipy.optimize.minimize to find the optimal weights
    result = minimize(objective_GMV, initial_weights, args=(V,), method='SLSQP',bounds=bounds, constraints=constraints)
    
    # Extract the optimal weights
    optimal_weights = result.x
    return optimal_weights

In [103]:
GMV_C_weights = pd.DataFrame(index = ret.index, columns =ret.columns)

for i in ret.index[60:]:#start from the 61th month
    start_date = i - pd.DateOffset(months = 60)
    end_date = i-pd.DateOffset(months = 1)
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()
    
    df = ret.loc[start_date:end_date]

    # calculate GMV-C portfolio for each rolling window
    GMV_C_weights.loc[i] = GMV_C_w(df)

GMV_C_weights=GMV_C_weights.dropna()

In [104]:
# Return on MVP-C portfolio with non-negative constraints:
MVP_C_return = (MVP_C_weights * ret.iloc[60:]).sum(axis = 1)

# Return on GMV-C portfolio with non-negative constraints:
GMV_C_return = (GMV_C_weights * ret.iloc[60:]).sum(axis = 1)

In [106]:
# Sharpe ratio for MVP-C portfolio with constraints:
print(f'Sharpe Ratio of MVP-C portfolio with non-negative constraints is {sharpe_ratio(MVP_C_return)}.')
# Sharpe ratio for GMV-C portfolio with constraints:
print(f'Sharpe Ratio of GMV-C portfolio with non-negative constraints is {sharpe_ratio(GMV_C_return)}.')

Sharpe Ratio of MVP-C portfolio with non-negative constraints is 0.6314021927686365.
Sharpe Ratio of GMV-C portfolio with non-negative constraints is 0.6790103409313136.
